<a href="https://colab.research.google.com/github/franzis17/EnronEmailAnalysis/blob/main/Business_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages


## Initialise all packages needed in the program

Packages used:
* calendar --> 
* matplotlib --> for visualisation purposes. Plot the data in to some kind of a chart.
* pandas --> for efficiently sorting different kinds of data (i.e. sorting messages data in a monthly order)
* sqlite3 --> for extracting data in the database and perform 3 different kinds of analysis on the data



In [ ]:
import calendar
import matplotlib.pyplot as plt
import pandas as pd
import re  # regular expression
import sqlite3
from wordcloud import WordCloud

# Generate Stopwords
from google.colab import output
!curl -Ol https://raw.githubusercontent.com/michael-borck/isys2001-worksheets/main/stopwords.py
output.clear()
print("Required packages installed")

## Connect to the database

In [ ]:
# Connect to the database using 'sqlite3'
conn = sqlite3.connect('/content/drive/MyDrive/a-quick_uploads/enron.db')

# Create a cursor to navigate each rows in the database and query any specific data from the dataset
cur = conn.cursor()

# <u>**Analysis 1.**</u> Email Traffic Over Time

Aim: Analyze the volume of emails sent over time by counting the number of messages of each employees sent per month.

## **Problem:**

* Must count the total number of messages of all employees they are sending per month.

## **Inputs:**

* List of messages

## **Outputs:**

* A line chart that shows the number of emails per month. (x = month, y = number of emails(messages))

## **Algorithm:**
  1. Create a dataframe based on an SQL statement that obtains all messages from the database table "message".
  2. Using the "message" dataframe, use the total amount of messages of the dataframe to sort the data monthly.
  3. Using the monthly-sorted messages data, create a line chart that shows the total amount of messages per month.

* **Important**: Must close the connection to the SQLite Database when done with the database.

## **Python Implementation:**

In [ ]:
'''
Note:
> Lines 12 to 29 were generated by ChatGPT. More info on reference list [ChatGPT-1]
'''

# Query database for all messages and to be put on a dataframe
sql='''
SELECT * FROM message
'''
messages_df = pd.read_sql(sql, conn)

# Make a new month table to group each messages by month
messages_df['date'] = pd.to_datetime(messages_df['date'])
messages_df['month'] = messages_df['date'].dt.month

# Apply month names instead of the month numbers
messages_df['month'] = messages_df['month'].apply(lambda x: calendar.month_name[x])
# Set month as a categorical column with desired order (Order the month chronologically)
month_order = [calendar.month_name[i] for i in range(1, 13)]
messages_df['month'] = pd.Categorical(messages_df['month'], categories=month_order, ordered=True)

monthly_counts = messages_df.groupby('month').size()
monthly_counts.plot(kind='line', figsize=(10, 6))

# Plot the total amount of messages sent per month
plt.xlabel('Month')
plt.ylabel('Number of messages')
plt.title('Total Number of Messages per Month in the year 2000')
plt.show()

# <u>**Analysis 2.**</u> Top Senders and Receivers
Aim: Identify the most frequent email senders and recipients by aggregating(collecting) the data in the 'Message' and 'RecepientInfo' tables.

# <u>**Analysis 3.**</u> Subject Keyword Analysis
Aim: Extract keywords from email subjects in the 'Message' table and analyze the frequency of words used to understand common topics of discussion.

## **Problem:**

* Must understand the common topics of discussion from the dataset of 'Message' table to see if it is related to any potential fraudulent activities.

## **Inputs:**

* List of words from the subject section of all email email messages. (database table located in message.subject)

## **Outputs:**

* WordCloud that shows the most common words used in the subject section of the email with. (Frequency of words depends on the size, i.e. large-sized words are most frequently used and smaller-sized are less frequent.)

## **Algorithm:**
  1. Extract all dataset of 'Message' table.
  2. Clean the 'subject' contents of the message dataset to only contain absolute words (words must not contain any symbols/numbers)
  3. Generate list of stop words called "ENGLISH_STOP_WORDS" from Stopwords.py, which is a list of words that do not provide much meaning to the analysis.
  4. Create a WordCloud using the "Cleaned-subject" list of words.

## **Python Implementation:**

Before running: Must run the Analysis 1 first to get the dataframe of messages "messages_df"

In [ ]:
from stopwords import ENGLISH_STOP_WORDS

# [Ref-2]
def clean(text):
    ''' Uses regular expresison to extract english letter and digits from the supplied text. '''
    regExp = "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
    return ' '.join(re.sub(regExp, " ", text).split())


# no need to extract messages from the database assuming it has been extracted from Analysis 1

# Create a new column as a placeholder for the clean subject keywords
messages_df['Clean Subject'] = messages_df['subject'].apply(clean)

# Add additional words to stopwords that do not have much meaning
additional_stop_words = {'New', 'Meeting', 'Update', 'Enron', 'Market', 'Power', 'Caiso', 'Notice', 'Hour', 'HourAhead', 'California', 'FW', 'Date', 'Fwd', 'Letter', 'Deal'}
ENGLISH_STOP_WORDS.update(additional_stop_words)

# Create and Display the wordcloud
subjects = ' '.join(messages_df['Clean Subject'])
wordcloud = WordCloud(width=680, height=480, margin=0, stopwords=ENGLISH_STOP_WORDS).generate(subjects)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.show()

## **Discussion:**

Below are some keywords that implies something:

* "Codesite":
  * Based from analysing the message body of all subjects with the keyword "Codesite", it looks like the company had some for of system that checks for any variances to 

## Close the connection to the database
**Note:** Use when done with the database

In [ ]:
conn.close()

# Reference List

* [1] https://pandas.pydata.org/docs/user_guide/10min.html#plotting
* [2] Clean function obtained from WK-9 tutorial and is used to remove symbols and numbers from words.

**ChatGPT**
* [1]
  * Purpose: To plot the frequency of messages per month to a line chart
  * Prompt: how can I use pandas package in python to plot the frequency of messages per month in a year, given the messages data are in a dataframe.